<a href="https://colab.research.google.com/github/carloscastillo10/kbs-scientific-publications/blob/development/notebooks/sparql-queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
from pandas import json_normalize

In [ ]:
repository_url = 'http://carloscastillo:7200/repositories/kbs-scientific-publications'
sparql = SPARQLWrapper(repository_url)

In [ ]:
def get_results(query, search_term):
    sparql.setQuery(query%(search_term))
    sparql.setReturnFormat(JSON)

    results = sparql.query().convert()
    return(results)

### Search of Text Similarity Index

In [ ]:
search_term = 'Ecuador'
query = """
    PREFIX : <http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst: <http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX pubo: <http://ontology.ontotext.com/publishing#>
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dct: <http://purl.org/dc/terms/>

    SELECT ?article ?articleID ?score 
    WHERE {
        ?search a inst:TextSimilarityIndex1;
                :searchTerm "%s";
                :searchParameters "";
                :documentResult ?result .
        ?result :value ?article ;
                :score ?score .
        ?article dct:identifier ?articleID .
    }
"""



results_df = json_normalize((get_results(query, search_term))['results']['bindings'])[[
    'article.value', 'articleID.value', 'score.value'
]]

results_df

,article.value,articleID.value,score.value
0,http://example.org/data/85116339194,85116339194,0.5453884454853958
1,http://example.org/data/85128084791,85128084791,0.5278737349903307
2,http://example.org/data/85115772290,85115772290,0.48837090494269664
3,http://example.org/data/85095945810,85095945810,0.47079868424039
4,http://example.org/data/85114870989,85114870989,0.46810150846004916
5,http://example.org/data/85128786779,85128786779,0.45637167568285053
6,http://example.org/data/85103395370,85103395370,0.4468280132316489
7,http://example.org/data/85115797925,85115797925,0.442704681499791
8,http://example.org/data/85128084791,85128084791,0.4231802195049405
9,http://example.org/data/85125949872,85125949872,0.4177475839732456


### Search of Initial Metadata Predication Index

In [ ]:
search_term = 'http://example.org/data/85095945810'
query = """
    PREFIX : <http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst: <http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX psi: <http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX pred: <http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT DISTINCT ?article ?id ?title ?abstract 
            (GROUP_CONCAT(DISTINCT ?author; SEPARATOR=", ") AS ?authors) ?score
    WHERE {
        ?search a inst:InitialMetadataPredicationIndex;
                psi:searchEntity <%s>;
                psi:searchPredicate <http://www.ontotext.com/graphdb/similarity/psi/any>;
                psi:entityResult ?result .
        ?result :value ?article;
                :score ?score .
        ?article rdf:type bibo:Article ;
                dct:identifier ?id .
        OPTIONAL { ?article dct:title ?title .}
        OPTIONAL { ?article bibo:abstract ?abstract .}
        OPTIONAL { 
            ?article dct:creator ?creator .
            OPTIONAL {
                ?creator foaf:givenName ?givenName;
                        foaf:familyName ?familyName .
            }
            BIND(CONCAT(STR(?givenName), " " ) AS ?firstName ) .
            BIND(CONCAT(STR(?firstName), STR(?familyName) ) AS ?author ) .
        }
    } GROUP BY ?article ?id ?title ?abstract ?score
"""

results_df = json_normalize((get_results(query, search_term))['results']['bindings'])[[
    'article.value', 'id.value', 'title.value', 'authors.value', 'score.value'
]]

results_df

,article.value,id.value,title.value,authors.value,score.value
0,http://example.org/data/85095945810,85095945810,The demand for a COVID-19 vaccine in Ecuador,"Carlos E. Carpio, Darren Hudson, Ivan Borja, O...",1.00000000496657
1,http://example.org/data/85125469862,85125469862,Promoting immunization equity in Latin America...,"Isabella L. Chan, Javier Roberti, Juan Pedro A...",0.28612400659157367


### Search of Linked Metadata Predication Index

In [ ]:
search_term = 'http://example.org/data/85126313252'
query = """
    PREFIX : <http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst: <http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX psi: <http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX pred: <http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX schema: <http://schema.org/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>


    SELECT DISTINCT ?article ?id ?title ?abstract 
        (GROUP_CONCAT(DISTINCT ?author; SEPARATOR=", ") AS ?authors) 
        (GROUP_CONCAT(DISTINCT ?mention; SEPARATOR=", ") AS ?mentions) 
        (GROUP_CONCAT(DISTINCT ?skosConcept; SEPARATOR=", ") AS ?skosConcepts) ?score
    WHERE {
        ?search a inst:LinkedMetadataPredicationIndex;
            psi:searchEntity <%s>;
            psi:searchPredicate <http://www.ontotext.com/graphdb/similarity/psi/any>;
            psi:entityResult ?result .
        ?result :value ?article;
                :score ?score .
        ?article rdf:type bibo:Article ;
                    dct:identifier ?id .
        OPTIONAL { ?article dct:title ?title . }
        OPTIONAL { ?article bibo:abstract ?abstract . }
        OPTIONAL { 
            ?article dct:creator ?creator .
            OPTIONAL {
                ?creator foaf:givenName ?givenName;
                        foaf:familyName ?familyName .
            }
            BIND(CONCAT(STR(?givenName), " " ) AS ?firstName ) .
            BIND(CONCAT(STR(?firstName), STR(?familyName) ) AS ?author ) .
        }
        OPTIONAL {
            ?article schema:mentions ?mention .
            OPTIONAL {?mention dct:subject ?skosConcept}    
        }
    } GROUP BY ?article ?id ?title ?abstract ?score
"""

results_df = json_normalize((get_results(query, search_term))['results']['bindings'])[[
    'article.value', 'id.value', 'title.value', 'authors.value', 
    'mentions.value', 'skosConcepts.value', 'score.value'
]]

results_df

,article.value,id.value,title.value,authors.value,mentions.value,skosConcepts.value,score.value
0,http://example.org/data/85126313252,85126313252,Responsible Marketing in the Traffic Light Lab...,"Cecilia Alexandra Fernández-Lucero, Cecilia Al...","http://dbpedia.org/resource/Ecuador, http://db...",http://dbpedia.org/resource/Category:Member_st...,1.0000000121236772
1,http://example.org/data/85106555867,85106555867,Tennis coaches’ perceptions of covid-19 impact...,"Jesus Ramón-Llin, Jesus Ramon-Llin, Jesús Ramó...","http://dbpedia.org/resource/Pandemic, http://d...",http://dbpedia.org/resource/Category:Global_he...,0.3134516501482811
2,http://example.org/data/85111508532,85111508532,SARS-CoV-2/DENV co-infection: a series of case...,"Ana Paula Monteiro Gomides Reis, Carlos Henriq...","http://dbpedia.org/resource/Brazil, http://dbp...",http://dbpedia.org/resource/Category:Member_st...,0.25636583799974555
3,http://example.org/data/85120685162,85120685162,Seroprevalence of anti-sars-cov-2 igg at the f...,"Henrik Salje, Simon Cauchemez, Anaïs Perilhou,...","http://dbpedia.org/resource/Protein, http://db...","http://dbpedia.org/resource/Category:Proteins,...",0.253861171038983
4,http://example.org/data/85121128473,85121128473,Peruvian small and medium-sized enterprises in...,"Mariano Martin Genaro Palacios Acuache, Susann...","http://dbpedia.org/resource/Pandemic, http://d...",http://dbpedia.org/resource/Category:Global_he...,0.24878730049726608
